In [1]:
from load_feat_pd import *


if __name__ == '__main__':
    
    base_folder_path = Path('/data/storage025/Turntaking/wavs_single_channel_normalized_nosil') if norm else Path('/data/storage025/Turntaking/wavs_single_channel_nosil')

    feats2level = {
        'jitter': 'utt',
        'shimmer': 'utt',
        'rp': 'utt',
        'f0': 'frame',
        'energy': 'frame'
    }

    np.set_printoptions(precision=2)
    allfeats = ['jitter', 'shimmer', 'rp', 'f0', 'energy']
    # for feat in allfeats:
    featname = 'f0'

    metadata = load_feat(base_folder_path, feature_name=featname)


Processing PictureNaming folder...
Found 1652 npy files
All value larger than 500.0 in /data/storage025/Turntaking/wavs_single_channel_normalized_nosil/PictureNaming-features/f0/subj-2112_rups.png_1.wav_f0.npy
All value larger than 500.0 in /data/storage025/Turntaking/wavs_single_channel_normalized_nosil/PictureNaming-features/f0/subj-2202_schatkist.png_1.wav_f0.npy
All value larger than 500.0 in /data/storage025/Turntaking/wavs_single_channel_normalized_nosil/PictureNaming-features/f0/subj-2127_rups.png_1.wav_f0.npy
All value larger than 500.0 in /data/storage025/Turntaking/wavs_single_channel_normalized_nosil/PictureNaming-features/f0/subj-2120_kaars.png_1.wav_f0.npy
0 files with all 0 values
Processing EarlyLate folder...
Found 3971 npy files
All value larger than 500.0 in /data/storage025/Turntaking/wavs_single_channel_normalized_nosil/EarlyLate-features/f0/subj-2112_41_L_gereedschap_tanden.wav_1.wav_f0.npy
All value larger than 500.0 in /data/storage025/Turntaking/wavs_single_chan

In [3]:
import pandas as pd
from pymer4.models import Lmer


def lme(metadata, featname='shimmer', level='utt', norm=True, log_feat=False, stats='mean'):

    df = pd.DataFrame(metadata)
    
    # remove group id == '11':
    df = df[df['group_id'] != '11']
    
    # map group_id 21 to HC 22 to PD
    df['group_id'] = df['group_id'].map({'21': 'HC', '22': 'PD'})


    # change age from string to int
    df['age'] = pd.to_numeric(df['age'], errors='coerce')
    missing_age = df['age'].isnull().sum()
    if missing_age > 0:
        print(f"warning 'age' has {missing_age} NaN, has been deleted")
        df = df.dropna(subset=['age'])
        
    # add one more column item name in the filename
    df['item'] = df['filename'].apply(lambda x: x.split('.')[0].split('_')[-1])



    if level == 'utt':
        # change the value from np array to float value
        df['value'] = df['value'].apply(lambda x: x[0])
        
    elif level == 'frame':
        if stats == 'mean':
            df['value'] = df['value'].apply(lambda x: x.mean())
        elif stats == 'std':
            df['value'] = df['value'].apply(lambda x: x.std())
        elif stats == 'median':
            df['value'] = df['value'].apply(lambda x: np.median(x))

    print(df.head())
    print(df.shape)



    model = Lmer("value ~ age + group_id*experiment + group_id*gender + (1|subject_id) + (1|item)", data=df)
                        

    result = model.fit()

    print(result)

    

if __name__ == '__main__':
    lme(metadata, featname=featname, level=feats2level[featname], norm=False, stats='std')



warning 'age' has 256 NaN, has been deleted
            experiment group_id      value subject_id  \
0  exp_1_PictureNaming       HC  15.851922       2118   
1  exp_1_PictureNaming       PD   3.084067       2203   
2  exp_1_PictureNaming       HC   2.287803       2124   
3  exp_1_PictureNaming       HC  11.462384       2106   
4  exp_1_PictureNaming       HC  40.665956       2129   

                           filename   age gender moca education       item  
0  subj-2118_wenkbrauw.png_1.wav_f0  71.0      V   30         7  wenkbrauw  
1      subj-2203_kaars.png_1.wav_f0  63.0      M   26         6      kaars  
2      subj-2124_kraan.png_1.wav_f0  79.0      V   25         7      kraan  
3       subj-2106_etui.png_1.wav_f0  75.0      V   26         6       etui  
4       subj-2129_etui.png_1.wav_f0  73.0      V   23         3       etui  
(7757, 10)
Linear mixed model fit by REML [’lmerMod’]
Formula: value~age+group_id*experiment+group_id*gender+(1|subject_id)

Family: gaussian	 Inferenc

/home/yzhong/data/storage025/yzhongenv/lib/python3.10/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
